# Load data and go to directory with python scripts

In [9]:
scripts_path = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts/'

# Path to save figures
path_to_save = '/'.join(scripts_path.split('/')[:-2]) + '/'

# Path to save supplementary and figure data folders
save_path = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/'
%cd $scripts_path

/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts


In [10]:
from dataLoader import *
from helper import *
import scipy.stats as st
from collections import Counter
import sklearn
from matplotlib import cm
import scipy
import  itertools
from datetime import datetime

from seaborn import clustermap
from scipy.cluster.hierarchy import linkage
from sklearn.linear_model import LogisticRegression
import os
import time
import pickle as pkl
import re
from matplotlib_venn import venn3, venn2
from statsmodels.stats.multitest import multipletests
dl = dataLoader(pt_perc = {'metabs': .25, '16s': .05, 'scfa': 0, 'toxin':0}, meas_thresh = 
                {'metabs': 0, '16s': 10, 'scfa': 0, 'toxin':0}, 
                var_perc = {'metabs': 15, '16s': 5, 'scfa': 0, 'toxin':0})


No patient demographic/clinical data available due to identifiability


# Define functions for doing over representation analysis

In [11]:
import math
def combination(n,k):
    return math.factorial(n)/(math.factorial(k)*math.factorial(n-k))

def ora(x, X):
    N = len(X)
    n = len(x)
#     print('n=' + str(n))
    num_hits = {}
    num_hits_expected = {}
    prob = {}
    for group in np.unique(x):
        k = np.sum(np.array(x)==group)
        K = np.sum(np.array(X)==group)
        if K < 4 or k < 3 or group == 'None' or group == 'Partially Characterized Molecules':
#             print(group + ': K='+str(K) + ', k=' + str(k))
            continue
        num_hits[group] = k
        num_hits_expected[group] = (K/N)*n
        prob[group] = (combination(K,k)*combination(N-K, n-k))/combination(N,n)
    if len(prob)==0:
        return None

    gps, pvals = zip(*prob.items())
    reject, corrected, a1, a2 = multipletests(list(pvals), alpha=.05, method='fdr_bh')
    fdr = {gp:corr for gp, corr in zip(gps, corrected)}
    out_df = pd.DataFrame({'# Hits':num_hits, '# Expected Hits': num_hits_expected, 'p-value': prob, 'FDR': fdr})
    return out_df.sort_values('p-value', ascending = True)

# Enrichment analysis for metabolites

In [18]:
path = path_to_save + '/univariate_analysis_control/metabs/'
save_dict = {}
keys = []
vals = []
file = 'OLS_metabs_Recurrer_0v1.csv'
df = pd.read_csv(path + file, index_col = 0)
mets = df.index.values[df['FDR, Week']<= 0.05]

In [22]:
pathway = 'SUPER_PATHWAY'
X = dl.col_mat_mets[pathway].loc[df.index.values]
x = dl.col_mat_mets[pathway].loc[mets]

In [48]:
group = 'Xenobiotics'
k = np.sum(np.array(x)==group)
K = np.sum(np.array(X)==group)

In [38]:
N = len(X)
n = len(x)

In [39]:
(combination(K,k)*combination(N-K, n-k))/combination(N,n)

0.1512927729792643

In [44]:
combination(N-K,n-k)

4.0941550940571055e+60

In [45]:
combination(K,k)*combination(N-K,n-k)

1.9571003005264597e+67

In [46]:
combination(N,n)

1.2935847905931988e+68

In [47]:
combination(K,k)

4780230.0

In [51]:
combination(218,3)

1703016.0

In [52]:
combination(105,4)

4780230.0

In [40]:
(K/N)*n

5.6644736842105265

In [41]:
n

41

In [49]:
K

218

In [50]:
k

3

In [12]:
# Set path to where univariate analysis results of metabolites is stored
path = path_to_save + '/univariate_analysis_control/metabs/'
save_dict = {}
keys = []
vals = []
for file in os.listdir(path):
    if '.csv' not in file:
        continue
    if 'ranksum' in file:
        continue
        
    df = pd.read_csv(path + file, index_col = 0)
    try:
        mets = df.index.values[df['FDR, Outcome']<= 0.05]
    except:
        mets = df.index.values[df['FDR, Week']<= 0.05]
    if 'rrer' in file:
        weeks = file.split('metabs')[1].split('_')[:-1]
        outcome = file.split('_')[-1].split('.')[0]
        a_type = 'Intra-group, ' + outcome + ', week ' + str(weeks[0]) + ' vs ' + str(weeks[1])
    else:
        a_type = 'Inter-group'

    for pathway in ['SUPER_PATHWAY','SUB_PATHWAY']:
        K = dl.col_mat_mets[pathway].loc[df.index.values]
        k = dl.col_mat_mets[pathway].loc[mets]
#         print(file)
#         print(pathway)
#         print(ora(k,K))
        val = ora(k,K)
        if val is not None:
            keys.append((a_type, pathway))
            vals.append(val.append(pd.Series(['']*val.shape[1], name = '_')))
#         save_dict['Metabolites'][(a_type, pathway)] = ora(k,K)
#         print('')
out_df_mets = pd.concat(vals, axis = 0, keys = keys)

In [15]:
out_df_mets

# Hits  \
Intra-group, 0v1, week  vs Recurrer SUPER_PATHWAY Xenobiotics              3.0   
                                                  Lipid                   25.0   
                                                  Nucleotide               5.0   
                                                  Peptide                  3.0   
                                                  Amino Acid               4.0   
...                                                                        ...   
Inter-group                         SUPER_PATHWAY Xenobiotics              5.0   
                                                  _                        NaN   
                                    SUB_PATHWAY   Sphingomyelins           4.0   
                                                  Food Component/Plant     4.0   
                                                  _                        NaN   

                                                                        # Expected Hits  \
Intra-group, 0v1, week  vs Recurrer SUPER_PATHWAY Xenobiotics                 11.760526   
                                                  Lipid                       15.968421   
                                                  Nucleotide                   2.319737   
                                                  Peptide                      1.456579   
                                                  Amino Acid                   5.664474   
...                                                                                 ...   
Inter-group                         SUPER_PATHWAY Xenobiotics                  6.023684   
                                                  _                                 NaN   
                                    SUB_PATHWAY   Sphingomyelins               0.386842   
                                                  Food Component/Plant         3.951316   
                                                  _                                 NaN   

                                                                         p-value  \
Intra-group, 0v1, week  vs Recurrer SUPER_PATHWAY Xenobiotics           0.000520   
                                                  Lipid                 0.001808   
                                                  Nucleotide            0.051493   
                                                  Peptide               0.121885   
                                                  Amino Acid            0.151293   
...                                                                          ...   
Inter-group                         SUPER_PATHWAY Xenobiotics           0.178518   
                                                  _                          NaN   
                                    SUB_PATHWAY   Sphingomyelins        0.000346   
                                                  Food Component/Plant  0.219939   
                                                  _                          NaN   

                                                                             FDR  \
Intra-group, 0v1, week  vs Recurrer SUPER_PATHWAY Xenobiotics           0.002601   
                                                  Lipid                 0.004519   
                                                  Nucleotide            0.085822   
                                                  Peptide               0.151293   
                                                  Amino Acid            0.151293   
...                                                                          ...   
Inter-group                         SUPER_PATHWAY Xenobiotics           0.178518   
                                                  _                          NaN   
                                    SUB_PATHWAY   Sphingomyelins        0.000691   
                                                  Food Component/Plant  0.219939   
                                                  _          

# Enrichment analysis for ASVs

In [13]:
path = path_to_save + '/univariate_analysis_control/16s/'
path_taxa = 'inputs/'
taxa = pd.read_csv(path_taxa + 'dada2-taxonomy-rdp.csv', index_col = 0)
save_dict['ASVs'] = {}
keys = []
vals = []
for file in os.listdir(path):
    if '.csv' not in file:
        continue
        
    df = pd.read_csv(path + file, index_col = 0)
    FAM = taxa[df.index.values].loc['Family'].dropna()
    mets = df.index.values[df['padj']<= 0.05]
    fam = taxa[mets].loc['Family'].dropna()
    val = ora(fam, FAM)
    
    if 'rrer' in file:
        weeks = file.split('rrer')[1].split('.')[0].split('_')
        outcome = file.split('_')[1].split('s')[1].split(weeks[0])[0]
        a_type = 'Intra-group, ' + outcome + ', week ' + str(weeks[0]) + ' vs ' + str(weeks[1])
    else:
        a_type = 'Inter-group'
        
    if val is not None:
        keys.append((a_type, 'Family'))
        vals.append(val.append(pd.Series(['']*val.shape[1], name = '_')))
out_df_ASVs = pd.concat(vals, axis = 0, keys = keys)

In [14]:
path_save = save_path + '/Supplemental Tables/'
fname = 'Enrichment Analysis - Controlled.xlsx'
with pd.ExcelWriter(path_save + fname, mode = 'w') as writer:
    out_df_ASVs.to_excel(writer, sheet_name = 'ASVs')
    
with pd.ExcelWriter(path_save + fname, mode = 'a') as writer:
    out_df_mets.to_excel(writer, sheet_name = 'Metabolites')